In [1]:
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [2]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    p.patient_code,
    b.first_name,
    b.last_name,
    b.dob,
    dsd.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    dgs.date AS session_date,
    dg.name AS group_name,
    dh.name AS hub_name,
    lc.name AS hub_commune,
    ld.name AS hub_departement,
    dsd.f_komin AS interview_commune,
    ldt.name AS topic,
    au.email as created_by,
    au.username,
    dgs.created_at as date_session_were_entered
FROM
    caris_db.dream_group_attendance dga
        LEFT JOIN
    dream_member dm ON dm.id_patient = dga.id_patient
        LEFT JOIN
    dreams_surveys_data dsd ON dsd.case_id = dm.case_id
        LEFT JOIN
    dream_group_session dgs ON dgs.id = dga.id_group_session
        LEFT JOIN
    dream_group dg ON dg.id = dgs.id_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
        LEFT JOIN
    lookup_dreams_topic ldt ON ldt.id = dgs.topic
        LEFT JOIN
    patient p ON p.id = dga.id_patient
        LEFT JOIN
    beneficiary b ON b.id_patient = dga.id_patient
        left join 
            auth_users au  on au.id=dgs.created_by
WHERE
    dga.value = 'P'
'''

evaluation_performance_attendance = pd.read_sql_query(query,engine,parse_dates=True)


# close the pool of connection
engine.dispose()

In [3]:
## get the test excel file from Query (au besoin de verification)
evaluation_performance_attendance.to_excel('epa.xlsx',index=False,na_rep="NULL")

In [4]:
evaluation_performance_attendance.shape

(385369, 15)

## Duplication aspect

In [5]:
evaluation_performance_attendance.dtypes

patient_code                         object
first_name                           object
last_name                            object
dob                                  object
interview_date                       object
session_date                         object
group_name                           object
hub_name                             object
hub_commune                          object
hub_departement                      object
interview_commune                    object
topic                                object
created_by                           object
username                             object
date_session_were_entered    datetime64[ns]
dtype: object

In [6]:
epa_deduplicate = evaluation_performance_attendance.drop_duplicates(subset=['patient_code'])

In [7]:
epa_deduplicate.shape

(22159, 15)

## Period of control aspect

In [8]:
 epa_startmars_endnow= epa_deduplicate[epa_deduplicate['date_session_were_entered']>='2021-03-01']

In [9]:
epa_startmars_endnow.shape

(4082, 15)

## Count by agents in descending order

In [10]:
performance_session_global  = epa_startmars_endnow.groupby(['created_by']).count()

In [11]:
performance_session_global.sort_values(ascending=False, by='patient_code',inplace=True)

In [12]:
performance_session  = pd.DataFrame( {"nombre_de_filles": performance_session_global['patient_code']})

In [13]:
performance_session

,nombre_de_filles
created_by,
billyjolicoeur95@gmail.com,1273
byronfabrice9@gmail.com,807
elionafredly7@gmail.com,721
blonnichel@yahoo.fr,472
ernst.belabre@carisfoundationintl.org,193
twoody103@gmail.com,149
francenelstjean.saed@yahoo.com,127
diegoarmando.dorlean@carisfoundationintl.org,100
gerald.poteau@carisfoundationintl.org,98


## MODIFY BELOW TO GET THE DIFFERENCE GIVEN LAST WEEK

In [14]:
week_before_perf = pd.read_excel("rapport_agents_saisie_des_sessions_7Juin2021.xlsx", sheet_name="filles_saisies_global", index_col="created_by")
performance_session["difference_par_rapport_a_la_semaine_derniere"] =  performance_session["nombre_de_filles"] - week_before_perf["nombre_de_filles"]
performance_session.difference_par_rapport_a_la_semaine_derniere = performance_session.difference_par_rapport_a_la_semaine_derniere.astype("int16")

performance_session

,nombre_de_filles,difference_par_rapport_a_la_semaine_derniere
created_by,,
billyjolicoeur95@gmail.com,1273,102
byronfabrice9@gmail.com,807,73
elionafredly7@gmail.com,721,178
blonnichel@yahoo.fr,472,0
ernst.belabre@carisfoundationintl.org,193,36
twoody103@gmail.com,149,1
francenelstjean.saed@yahoo.com,127,2
diegoarmando.dorlean@carisfoundationintl.org,100,0
gerald.poteau@carisfoundationintl.org,98,5


## Agent per Commune

In [15]:
performance_agents_par_commune = epa_startmars_endnow.pivot_table(index='created_by',columns='hub_commune',values=['patient_code'],aggfunc='count',fill_value=0,margins=True, margins_name="Grand Total")
#performance_agents_par_commune.reset_index(inplace=True)

In [16]:
performance_agents_par_commune.sort_values(by=('patient_code',"Grand Total") ,ascending=False,inplace=True)

In [17]:
performance_agents_par_commune

patient_code                   \
hub_commune                                   Cap-Haïtien Carrefour Delmas   
created_by                                                                   
Grand Total                                           279       270    164   
billyjolicoeur95@gmail.com                            139        72    123   
byronfabrice9@gmail.com                                70        63      0   
elionafredly7@gmail.com                                31        90     39   
blonnichel@yahoo.fr                                     0         0      0   
ernst.belabre@carisfoundationintl.org                   0         0      0   
twoody103@gmail.com                                    18        45      2   
francenelstjean.saed@yahoo.com                          0         0      0   
diegoarmando.dorlean@carisfoundationintl.org            0         0      0   
gerald.poteau@carisfoundationintl.org                   0         0      0   
jhonip77@yahoo.fr                                      21         0      0   
roselord.despeignes@carisfoundationintl.org             0         0      0   

                                                                  \
hub_commune                                  Desdunes Dessalines   
created_by                                                         
Grand Total                                        62        179   
billyjolicoeur95@gmail.com                         18          0   
byronfabrice9@gmail.com                             0         58   
elionafredly7@gmail.com                             0          0   
blonnichel@yahoo.fr                                 0          0   
ernst.belabre@carisfoundationintl.org              36          0   
twoody103@gmail.com                                 0          0   
francenelstjean.saed@yahoo.com                      2        121   
diegoarmando.dorlean@carisfoundationintl.org        0          0   
gerald.poteau@carisfoundationintl.org               5          0   
jhonip77@yahoo.fr                                   0          0   
roselord.despeignes@carisfoundationintl.org         1          0   

                                                                              \
hub_commune                                  Grande Saline Gressier Kenscoff   
created_by                                                                     
Grand Total                                            121      537      535   
billyjolicoeur95@gmail.com                               0       72      201   
byronfabrice9@gmail.com                                  0      161      215   
elionafredly7@gmail.com                                  0      273       64   
blonnichel@yahoo.fr                                      0        0        0   
ernst.belabre@carisfoundationintl.org                   22        0        0   
twoody103@gmail.com                                      0       31        1   
francenelstjean.saed@yahoo.com                           4        0        0   
diegoarmando.dorlean@carisfoundationintl.org            59        0        0   
gerald.poteau@carisfoundationintl.org                   36        0        0   
jhonip77@yahoo.fr                                        0        0       54   
roselord.despeignes@carisfoundationintl.org              0        0        0   

                                                                    ...  \
hub_commune                                  La Chapelle Liancourt  ...   
created_by                                                          ...   
Grand Total                                          130        40  ...   
billyjolicoeur95@gmail.com                             0         0  ...   
byronfabrice9@gmail.com                               37         0  ...   
elionafredly7@gmail.com                                0         0  ...   
blonnichel@yahoo.fr                                   50        40  ...   
ernst.belabre@carisfoundationintl.org               

## Reporting Performance de mars 2021 a date

In [18]:
pac = pd.ExcelWriter('rapport_agents_saisie_des_sessions.xlsx',engine='xlsxwriter')
performance_session.to_excel(pac,sheet_name='filles_saisies_global')
performance_agents_par_commune.to_excel(pac,sheet_name="filles_saisies_par_communes")
pac.save()